# Pruebas de los módulos NLP

Ejecuta cada componente de `src/nlp` contra los artículos ubicados en `data_example/`, incorporando las etiquetas y clasificaciones generadas directamente sobre los archivos `.json` originales para que el vectorizador los tenga disponibles antes de la integración con el resto del sistema.


In [ ]:
from pathlib import Path
import json
from pprint import pprint

from preprocessing import detect_language

PROJECT_ROOT = Path("/home/lia/Escritorio/Proyectos/NLP/Report_Generator/Data/Data_articles")
DATA_DIR = PROJECT_ROOT / "Data_articles4"

article_paths = sorted(DATA_DIR.glob("article_*.json"))


def load_article(path: Path) -> dict:
    with open(path, encoding="utf-8") as fh:
        return json.load(fh)


def persist_article(path: Path, data: dict) -> None:
    with open(path, "w", encoding="utf-8") as fh:
        json.dump(data, fh, ensure_ascii=False, indent=2)


def update_article(path: Path, updates: dict) -> dict:
    data = load_article(path)
    data.update(updates)
    persist_article(path, data)
    return data

sample = load_article(article_paths[0])
print(f"Artículos detectados: {len(article_paths)}")
print("Idioma detectado para el primero:", detect_language(sample.get("text", "")))


Artículos detectados: 9
Idioma detectado para el primero: es


In [12]:
from preprocessing import TextPreprocessor

preprocessor = TextPreprocessor(use_spacy=False, remove_stopwords=True)
preprocessing_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    processed = preprocessor.preprocess_full(text)
    updates = {
        "preprocessing": {
            "cleaned": processed["cleaned"],
            "tokens": processed["tokens"],
            "token_count": processed["token_count"],
            "sentence_count": processed["sentence_count"]
        }
    }
    update_article(path, updates)
    preprocessing_results[path.name] = {
        "token_count": updates["preprocessing"]["token_count"],
        "sentence_count": updates["preprocessing"]["sentence_count"]
    }

pprint(preprocessing_results.get("article_1.json"))


{'sentence_count': 9, 'token_count': 237}


In [13]:
import nltk
nltk.download('averaged_perceptron_tagger_eng')
from pos_analyzer import POSAnalyzer

pos_analyzer = POSAnalyzer()
pos_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    pos_info = pos_analyzer.analyze(text)
    patterns = pos_analyzer.get_top_patterns(pos_info, n=5)
    updates = {
        "pos_analysis": {
            "tag_freq": pos_info["tag_freq"],
            "bigram_freq": pos_info["bigram_freq"],
            "trigram_freq": pos_info["trigram_freq"],
            "top_patterns": patterns
        }
    }
    update_article(path, updates)
    pos_results[path.name] = patterns

pprint(pos_results.get("article_1.json"))


{'top_bigrams': {'FW-FW': 48,
                 'FW-NN': 16,
                 'JJ-NN': 16,
                 'NN-NN': 77,
                 'NNP-NN': 12},
 'top_tags': {'FW': 78, 'JJ': 39, 'NN': 126, 'NNP': 32, 'NNS': 13},
 'top_trigrams': {'FW-FW-FW': 33,
                  'FW-NN-FW': 8,
                  'JJ-NN-NN': 9,
                  'NN-NN-NN': 54,
                  'NNP-NN-NN': 10}}


[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/lia/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


In [14]:
from grammar_analyzer import GrammarAnalyzer

grammar_analyzer = GrammarAnalyzer()
grammar_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    grammar_info = grammar_analyzer.analyze(text)
    update_article(path, {"grammar_analysis": grammar_info})
    grammar_results[path.name] = {
        "chunk_counts": grammar_info["chunk_counts"],
        "top_rules": grammar_info["top_rules"][:5]
    }

pprint(grammar_results.get("article_1.json"))


{'chunk_counts': {'ADJP': 7, 'NP': 71, 'PP': 6, 'VP': 14},
 'top_rules': [{'frequency': 4,
                'rule': "PP -> ('de', 'IN') NP",
                'weight': 0.0374},
               {'frequency': 2,
                'rule': "NP -> ('ambas', 'NN') ('naciones', 'NNS')",
                'weight': 0.0187},
               {'frequency': 2,
                'rule': "ADJP -> ('Pezeshkian', 'JJ')",
                'weight': 0.0187},
               {'frequency': 2,
                'rule': "PP -> ('en', 'IN') NP",
                'weight': 0.0187},
               {'frequency': 1,
                'rule': "S -> NP VP ('de', 'FW') NP (',', ',') NP (',', ',') "
                        "NP NP ('solidaridad', 'FW') NP ('las', 'FW') VP PP "
                        "('en', 'FW') NP (',', ',') NP ('del', 'FW') NP ('.', "
                        "'.')",
                'weight': 0.0093}]}


In [15]:
from regex_annotator import RegexAnnotator

regex_annotator = RegexAnnotator()
regex_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    annotations = regex_annotator.annotate(text)
    update_article(path, {"regex_annotations": annotations})
    regex_results[path.name] = annotations["categories"]

pprint(regex_results.get("article_1.json"))
pprint(regex_results.get("article_11.json"))
pprint(regex_results.get("article_24.json"))


['CONFLICTOS_ARMADOS_OPERACIONES',
 'PAISES',
 'INTEGRACION_LATINOAMERICANA',
 'VIOLENCIA_CRIMEN_DERECHOS_HUMANOS',
 'CIUDADES_CAPITALES',
 'REGIONES_PROVINCIAS',
 'RELACIONES_INTERNACIONALES',
 'PROTESTAS_CONFLICTOS_SOCIALES',
 'LEYES_NORMATIVAS',
 'GOBIERNOS_OFICIALES',
 'ELECCIONES_PROCESOS_POLITICOS',
 'APROBACION_RECHAZO']
['CULTURA_DEPORTE_SOCIEDAD',
 'CONFLICTOS_ARMADOS_OPERACIONES',
 'PAISES',
 'INTEGRACION_LATINOAMERICANA',
 'VIOLENCIA_CRIMEN_DERECHOS_HUMANOS',
 'JURIDICO_LEGISLATIVO_CORRUPCION',
 'ORGANISMOS_INTERNACIONALES',
 'ACCIONES_LEGALES',
 'RELACIONES_INTERNACIONALES',
 'PROTESTAS_CONFLICTOS_SOCIALES',
 'CIUDADES_CAPITALES',
 'REGIONES_PROVINCIAS',
 'ANTIIMPERIALISMO_SOBERANIA',
 'ELECCIONES_PROCESOS_POLITICOS',
 'APROBACION_RECHAZO']
None


In [16]:
from relation_extractor import RelationExtractor

try:
    relation_extractor = RelationExtractor()
except ValueError as exc:
    relation_extractor = None
    print(f"No se pueden extraer relaciones: {exc}")

relation_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    if relation_extractor:
        rel_info = relation_extractor.extract(text)
    else:
        rel_info = {"entidades": [], "relaciones": [], "error": "Modelo spaCy no disponible"}
    update_article(path, {"knowledge_graph": rel_info})
    relation_results[path.name] = rel_info

pprint(relation_results.get("article_1.json"))


{'entidades': [{'nombre': 'Gobierno', 'tipo': 'Lugar'},
               {'nombre': 'Maduro', 'tipo': 'Persona'},
               {'nombre': 'Nicolás Maduro', 'tipo': 'Persona'},
               {'nombre': 'Teherán', 'tipo': 'Lugar'},
               {'nombre': 'Masoud Pezeshkian', 'tipo': 'Persona'},
               {'nombre': 'Washington', 'tipo': 'Lugar'},
               {'nombre': 'Washington', 'tipo': 'Persona'},
               {'nombre': 'Estados Unidos', 'tipo': 'Lugar'},
               {'nombre': 'América', 'tipo': 'Lugar'},
               {'nombre': 'Venezuela', 'tipo': 'Lugar'},
               {'nombre': 'Caracas', 'tipo': 'Lugar'},
               {'nombre': 'Estado', 'tipo': 'Organizacion'},
               {'nombre': 'Gobierno iraní', 'tipo': 'Lugar'},
               {'nombre': 'Pezeshkian', 'tipo': 'Persona'},
               {'nombre': 'Irán', 'tipo': 'Lugar'},
               {'nombre': 'el Caribe', 'tipo': 'Lugar'},
               {'nombre': 'Donald Trump', 'tipo': 'Persona'}],


In [17]:
from sentiment_analyzer import SentimentAnalyzer

sentiment_analyzer = SentimentAnalyzer()
sentiment_results = {}

for path in article_paths:
    article = load_article(path)
    text = article.get("text", "")
    if not text:
        continue
    sentiment = sentiment_analyzer.analyze(text)
    update_article(path, {"sentiment": sentiment})
    sentiment_results[path.name] = sentiment

pprint(sentiment_results.get("article_1.json"))


{'compound': 0.34, 'label': 'positive', 'neg': 0.0, 'neu': 0.993, 'pos': 0.007}
